In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter

# Set up plotting parameters
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14

def load_and_process_data():
    """Load real data or generate mock data for demonstration"""
    try:
        # Try to load real data
        demand_data = pd.read_csv("../demand_get/data/get1/demand_60.15269_-45.27849.csv")
        system_output = pd.read_csv('../result/output_0/60.15269_-45.27849_results.csv', 
                                   usecols=['WT', 'PV', 'WEC'])
        print(f"✓ Loaded real data:")
        print(f"  - Demand data: {len(demand_data)} hours (1-hour resolution)")
        print(f"  - System output: {len(system_output)} periods (3-hour resolution)")
        
        # Get heating demand (try different possible column names)
        demand_columns = ['heating_demand', 'Heating_Demand', 'demand', 'Demand']
        demand_col = None
        for col in demand_columns:
            if col in demand_data.columns:
                demand_col = col
                break
        
        if demand_col is None and len(demand_data.columns) > 0:
            demand_col = demand_data.columns[0]
            print(f"Using first available column as demand: {demand_col}")
        
        heating_demand = demand_data[demand_col].values
        
    except FileNotFoundError:
        print("📄 Real data files not found. Generating mock data for demonstration...")
        # Generate one full year of data
        # Demand: hourly resolution (8760 hours)
        hours_per_year = 8760
        periods_per_year = hours_per_year // 3  # 2920 3-hour periods
        
        # Generate hourly demand data
        time_index_hourly = np.arange(hours_per_year)
        seasonal_demand = 400 + 300 * np.cos(2 * np.pi * (time_index_hourly - 2160) / hours_per_year)
        daily_variation = 100 * np.sin(2 * np.pi * time_index_hourly / 24)
        heating_demand = np.maximum(100, seasonal_demand + daily_variation + 50 * np.random.randn(hours_per_year))
        
        # Generate 3-hourly system output data
        time_index_3h = np.arange(periods_per_year)
        time_hours_3h = time_index_3h * 3  # Convert to equivalent hours
        
        # Wind power: higher in winter, variable
        wind_seasonal = 200 + 150 * np.cos(2 * np.pi * (time_hours_3h - 1440) / hours_per_year)
        wind_variation = 100 * np.random.randn(periods_per_year)
        wt_output = np.maximum(0, wind_seasonal + wind_variation)
        
        # Solar PV: higher in summer, with 3-hour averages
        solar_seasonal = 150 * np.maximum(0, np.sin(2 * np.pi * (time_hours_3h - 4380) / hours_per_year))
        # Simulate day/night cycle for 3-hour periods
        solar_daily = 200 * np.maximum(0, np.sin(2 * np.pi * time_hours_3h / 24))
        pv_output = np.maximum(0, solar_seasonal + solar_daily + 30 * np.random.randn(periods_per_year))
        
        # WEC power: more consistent
        wec_base = 120 + 50 * np.cos(2 * np.pi * time_hours_3h / hours_per_year)
        wec_output = np.maximum(20, wec_base + 40 * np.random.randn(periods_per_year))
        
        system_output = pd.DataFrame({
            'WT': wt_output,
            'PV': pv_output, 
            'WEC': wec_output
        })
        
        print(f"✓ Generated mock data:")
        print(f"  - Demand data: {len(heating_demand)} hours (1-hour resolution)")
        print(f"  - System output: {len(system_output)} periods (3-hour resolution)")
    
    return heating_demand, system_output

def resample_demand_to_3hour(hourly_demand):
    """Resample hourly demand data to 3-hour resolution by taking mean"""
    n_hours = len(hourly_demand)
    n_3hours = n_hours // 3
    
    # Reshape and take mean of every 3 hours
    reshaped = hourly_demand[:n_3hours * 3].reshape(-1, 3)
    demand_3h = np.mean(reshaped, axis=1)
    
    print(f"🔄 Resampled demand from {n_hours} hours to {len(demand_3h)} 3-hour periods")
    return demand_3h

def smooth_data(data, window_periods=7*8):
    """Apply Savitzky-Golay smoothing (window in 3-hour periods)"""
    # For 3-hour data: 7 days = 7*8 = 56 periods
    window_3h = max(3, window_periods)
    if window_3h % 2 == 0:  # Ensure odd window size
        window_3h += 1
    
    polyorder = min(2, window_3h - 1)
    
    if isinstance(data, pd.DataFrame):
        smoothed = data.copy()
        for col in data.columns:
            smoothed[col] = np.maximum(0, savgol_filter(data[col], window_3h, polyorder))
        return smoothed
    else:
        return savgol_filter(data, window_3h, polyorder)

def create_monthly_ticks(n_periods):
    """Create month-based x-axis ticks for 3-hour resolution data"""
    # For 3-hour resolution: 8 periods per day, ~240 periods per month
    periods_per_month = [
        31*8, 28*8, 31*8, 30*8, 31*8, 30*8,  # Jan-Jun
        31*8, 31*8, 30*8, 31*8, 30*8, 31*8   # Jul-Dec
    ]
    
    month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    # Calculate cumulative positions for month boundaries
    cumulative_periods = np.cumsum([0] + periods_per_month)
    
    # Only include months that fit within our data
    valid_ticks = []
    valid_labels = []
    
    for i, (pos, label) in enumerate(zip(cumulative_periods, month_labels)):
        if pos < n_periods:
            valid_ticks.append(pos)
            valid_labels.append(label)
    
    return valid_ticks, valid_labels

def plot_energy_profile(heating_demand, system_output):
    """Create the main energy supply and demand plot"""
    
    # Resample demand from hourly to 3-hour resolution
    print("🔄 Processing data resolutions...")
    demand_3h = resample_demand_to_3hour(heating_demand)
    
    # Ensure both datasets have the same length
    min_length = min(len(demand_3h), len(system_output))
    demand_3h = demand_3h[:min_length]
    output_3h = system_output.iloc[:min_length].copy()
    
    print(f"✓ Final data length: {min_length} 3-hour periods")
    
    # Apply smoothing to both demand and output
    print("📈 Applying smoothing...")
    smooth_demand = smooth_data(demand_3h)
    smooth_output = smooth_data(output_3h)
    
    # Create stacked data for area plot
    stacked_data = pd.DataFrame(index=smooth_output.index)
    stacked_data['WEC'] = smooth_output['WEC']  # Bottom layer
    stacked_data['WT'] = stacked_data['WEC'] + smooth_output['WT']  # WEC + WT  
    stacked_data['PV'] = stacked_data['WT'] + smooth_output['PV']   # WEC + WT + PV
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(14, 6), dpi=300)
    
    # Define colors
    colors = {
        'WEC': '#1f77b4',      # Blue for Wave Energy Converter
        'WT': '#87ceeb',       # Light blue for Wind Turbine  
        'PV': '#ffcc00',       # Gold for Solar PV
        'Heating': '#d62728'   # Red for Heating Demand
    }
    
    # Time axis (3-hour periods)
    time_axis = np.arange(len(smooth_output))
    
    # Plot stacked areas
    ax.fill_between(time_axis, 0, stacked_data['WEC'], 
                   color=colors['WEC'], alpha=0.8, label='Wave Energy Converter (WEC)')
    ax.fill_between(time_axis, stacked_data['WEC'], stacked_data['WT'],
                   color=colors['WT'], alpha=0.8, label='Wind Turbine (WT)')
    ax.fill_between(time_axis, stacked_data['WT'], stacked_data['PV'],
                   color=colors['PV'], alpha=0.8, label='Solar PV')
    
    # Add boundary lines for clarity
    ax.plot(time_axis, stacked_data['WEC'], color='white', linewidth=0.5, alpha=0.7)
    ax.plot(time_axis, stacked_data['WT'], color='white', linewidth=0.5, alpha=0.7)
    ax.plot(time_axis, stacked_data['PV'], color='white', linewidth=0.5, alpha=0.7)
    
    # Plot heating demand line
    ax.plot(time_axis, smooth_demand, color=colors['Heating'], 
           linewidth=3, label='Heating Demand', zorder=5)
    
    # Set up x-axis with monthly labels
    month_ticks, month_labels = create_monthly_ticks(len(time_axis))
    ax.set_xticks(month_ticks)
    ax.set_xticklabels(month_labels)
    
    # Labels and formatting
    ax.set_xlabel('Month', fontsize=16, fontweight='bold')
    ax.set_ylabel('Power (kW)', fontsize=16, fontweight='bold')
    # ax.set_title('Annual Renewable Energy Supply vs. Heating Demand Profile\n(3-Hour Resolution)', 
    #             fontsize=16, fontweight='bold', pad=20)
    ax.xaxis.set_tick_params(labelsize=16)
    ax.yaxis.set_tick_params(labelsize=16)
    # Set axis limits
    y_max = max(stacked_data['PV'].max(), smooth_demand.max()) * 1.1
    ax.set_ylim(0, y_max)
    ax.set_xlim(0, len(time_axis) - 1)
    
    # Add legend
    ax.legend(loc='upper right', fontsize=16, frameon=True, fancybox=True, shadow=False)
    
    # Grid for better readability
    ax.grid(True, alpha=0.3, linestyle='--')
    
    # Remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    
    # Print some statistics
    total_renewable = stacked_data['PV'].iloc[-1] if len(stacked_data) > 0 else 0
    avg_demand = smooth_demand.mean() if len(smooth_demand) > 0 else 0
    
    print(f"\n📊 Statistics (3-hour resolution):")
    print(f"   Average renewable generation: {total_renewable:.1f} kW")  
    print(f"   Average heating demand: {avg_demand:.1f} kW")
    print(f"   Data periods: {len(time_axis)} (3-hour periods)")
    if avg_demand > 0:
        print(f"   Renewable coverage: {(total_renewable/avg_demand*100):.1f}%")
    
    plt.show()
    
    return fig, ax

# Main execution
print("🚀 Loading and processing energy data...")
heating_demand, system_output = load_and_process_data()

print("🎨 Creating visualization...")
fig, ax = plot_energy_profile(heating_demand, system_output)

print("✅ Visualization complete!")

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from geopy.distance import great_circle
import matplotlib.patches as mpatches

def load_island_lng_data():
    """Load island and LNG terminal data"""
    print("🏝️ Loading island data...")
    # Read island data
    islands = pd.read_csv("../visualization/filtered_island_1898.csv")
    # Clean column names and extract coordinates
    islands.columns = [col.strip() for col in islands.columns]
    if 'Long' in islands.columns and 'Lat' in islands.columns:
        islands['lon'] = islands['Long']
        islands['lat'] = islands['Lat']
    print(f"✓ Loaded {len(islands)} islands")
    
    print("⛽ Loading LNG terminal data...")
    # Read LNG terminal data
    lng_terminals = pd.read_excel("../visualization/LNG_Terminals.xlsx")
    # Filter only operational terminals with valid coordinates
    lng_active = lng_terminals[
        (lng_terminals['Status'].isin(['Operating', 'Under Construction'])) &
        (lng_terminals['Latitude'].notna()) & 
        (lng_terminals['Longitude'].notna())
    ].copy()
    lng_active['lon'] = lng_active['Longitude'] 
    lng_active['lat'] = lng_active['Latitude']
    print(f"✓ Loaded {len(lng_active)} active LNG terminals")
    
    return islands, lng_active

def calculate_min_distances(islands, lng_terminals):
    """Calculate minimum great circle distance from each island to nearest LNG terminal"""
    print("📏 Calculating distances to nearest LNG terminals...")
    
    min_distances = []
    
    for idx, island in islands.iterrows():
        island_coord = (island['lat'], island['lon'])
        
        # Calculate distance to all LNG terminals
        distances = []
        for _, terminal in lng_terminals.iterrows():
            terminal_coord = (terminal['lat'], terminal['lon'])
            # Calculate great circle distance in km and multiply by 1.2
            distance = great_circle(island_coord, terminal_coord).kilometers * 1.2
            distances.append(distance)
        
        # Find minimum distance
        min_distance = min(distances) if distances else float('inf')
        min_distances.append(min_distance)
        
        if idx % 500 == 0:
            print(f"  Processed {idx}/{len(islands)} islands...")
    
    islands['min_lng_distance'] = min_distances
    print(f"✓ Distance calculation complete. Range: {min(min_distances):.1f} - {max(min_distances):.1f} km")
    
    return islands

def visualize_island_lng_distances(islands, lng_terminals):
    """Create global map visualization with distance-based color coding"""
    plt.rcParams['font.family'] = 'Times New Roman'
    
    # Create Point geometries for islands
    islands['geometry'] = islands.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    geo_islands = gpd.GeoDataFrame(islands, geometry='geometry')
    geo_islands.set_crs(epsg=4326, inplace=True)
    
    # Create the figure with Robinson projection
    fig = plt.figure(figsize=(16, 8), dpi=300)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())

    # Set background and map features
    ax.set_facecolor('#FFFFFF')  
    ax.add_feature(cfeature.LAND, color="#CECECE", alpha=0.4)
    ax.add_feature(cfeature.OCEAN, color="#FFFFFF", alpha=0.5)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.7)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, alpha=0.3)
    
    # Color mapping for islands based on distance to LNG
    # Farther from LNG terminals = darker color
    sci_colors = ['#012f48', '#669aba', '#fbf0d9', '#be1420', '#7a0101']
    custom_cmap = mpl.colors.LinearSegmentedColormap.from_list('sci_palette', sci_colors)
    
    # Set distance range for color mapping
    max_distance = np.percentile(geo_islands['min_lng_distance'], 95)  # Use 95th percentile to avoid outliers
    norm = mpl.colors.Normalize(vmin=0, vmax=max_distance)
    
    # Point sizes
    min_size = 20
    max_size = 200
    
    # Sort islands by distance (closest first, so farthest are on top)
    geo_islands = geo_islands.sort_values(by='min_lng_distance')
    
    print("🏝️ Drawing islands...")
    # Draw island points
    for idx, row in geo_islands.iterrows():
        lon, lat = row.geometry.x, row.geometry.y
        distance = row['min_lng_distance']
        
        # Calculate size and color
        size_factor = min(distance / max_distance, 1.0)
        size = min_size + (size_factor * (max_size - min_size))
        color = custom_cmap(norm(distance))
        
        # Create point image
        temp_fig = plt.figure(figsize=(1, 1), frameon=False, dpi=100)
        temp_fig.patch.set_alpha(0)
        
        temp_ax = temp_fig.add_subplot(111)
        temp_ax.set_aspect('equal')
        temp_ax.patch.set_alpha(0)
        
        # Draw circle with black border
        outer_circle = plt.Circle((0.5, 0.5), 0.18, color='black', alpha=1)
        temp_ax.add_patch(outer_circle)
        
        inner_circle = plt.Circle((0.5, 0.5), 0.15, color=color, alpha=1)
        temp_ax.add_patch(inner_circle)
        
        temp_ax.set_xlim(0, 1)
        temp_ax.set_ylim(0, 1)
        temp_ax.axis('off')
        
        temp_fig.tight_layout(pad=0)
        temp_fig.canvas.draw()
        point_img = np.array(temp_fig.canvas.renderer.buffer_rgba())
        plt.close(temp_fig)
        
        # Transform coordinates and add to map
        x, y = ax.projection.transform_point(lon, lat, src_crs=ccrs.PlateCarree())
        zoom_factor = np.sqrt(size) / 50
        
        imagebox = OffsetImage(point_img, zoom=zoom_factor)
        imagebox.image.axes = ax
        
        ab = AnnotationBbox(imagebox, (x, y), frameon=False, pad=0, zorder=10)
        ax.add_artist(ab)
    
    print("⛽ Drawing LNG terminals...")
    # Draw LNG terminals as squares
    for _, terminal in lng_terminals.iterrows():
        lon, lat = terminal['lon'], terminal['lat']
        
        # Create square marker for LNG terminals
        temp_fig = plt.figure(figsize=(1, 1), frameon=False, dpi=100)
        temp_fig.patch.set_alpha(0)
        
        temp_ax = temp_fig.add_subplot(111)
        temp_ax.set_aspect('equal')
        temp_ax.patch.set_alpha(0)
        
        # Draw square
        square = plt.Rectangle((0.3, 0.3), 0.4, 0.4, color="#D2AB7B", alpha=1)
        temp_ax.add_patch(square)
        
        temp_ax.set_xlim(0, 1)
        temp_ax.set_ylim(0, 1)
        temp_ax.axis('off')
    
        temp_fig.tight_layout(pad=0)
        temp_fig.canvas.draw()
        terminal_img = np.array(temp_fig.canvas.renderer.buffer_rgba())
        plt.close(temp_fig)
        
        # Transform coordinates and add to map
        x, y = ax.projection.transform_point(lon, lat, src_crs=ccrs.PlateCarree())
        zoom_factor = 0.12
        
        imagebox = OffsetImage(terminal_img, zoom=zoom_factor)
        imagebox.image.axes = ax
        
        ab = AnnotationBbox(imagebox, (x, y), frameon=False, pad=0, zorder=5)
        ax.add_artist(ab)
    
    # Create legend for islands and LNG terminals
    island_patch = mpatches.Patch(color='#012f48', label='Islands (color by distance to LNG)')
    lng_patch = mpatches.Patch(color='#D2AB7B', label='LNG Terminals')
    
    # Add legend
    ax.legend(handles=[island_patch, lng_patch], 
              loc='lower left', 
              bbox_to_anchor=(0.02, 0.02), 
              frameon=True,
              fontsize=12,
            #   title='Legend',
              title_fontsize=12)
    
    # Create colorbar for island distances
    fake_scatter = ax.scatter([-1000], [-1000], c=[0], cmap=custom_cmap, 
                             vmin=0, vmax=max_distance, s=1)
    
    cbar = fig.colorbar(fake_scatter, ax=ax, orientation='horizontal', 
                       shrink=0.6, pad=0.05, aspect=50)
    cbar.set_label('Distance to Nearest LNG Terminal (km)', fontsize=14)
    
    # Add title and finalize
    # ax.set_title('Global Islands: Distance to Nearest LNG Terminals\n(Darker = Farther from LNG)', 
    #             fontsize=16, fontweight='bold', pad=20)
    fig.text(0.42, 0.05, 'Global Islands: Distance to Nearest LNG Terminals\n(Darker = Farther from LNG)', 
                    fontsize=9, style='italic', alpha=0.7)
    ax.set_global()
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\n📊 Distance Statistics:")
    print(f"   Average distance to LNG: {geo_islands['min_lng_distance'].mean():.1f} km")
    print(f"   Median distance to LNG: {geo_islands['min_lng_distance'].median():.1f} km")
    print(f"   Islands > 1000 km from LNG: {len(geo_islands[geo_islands['min_lng_distance'] > 1000])}")
    print(f"   Islands > 5000 km from LNG: {len(geo_islands[geo_islands['min_lng_distance'] > 5000])}")

# Main execution
print("🚀 Starting global island-LNG distance analysis...")
islands, lng_terminals = load_island_lng_data()
islands_with_distances = calculate_min_distances(islands, lng_terminals)
visualize_island_lng_distances(islands_with_distances, lng_terminals)
print("✅ Analysis complete!")

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.patches as mpatches
from collections import defaultdict
import os

plt.rcParams['font.family'] = 'Times New Roman'

# 1. 读取岛屿数据
filtered_islands = pd.read_csv('../demand_get/filtered_island_1898.csv')
print(f"读取到 {len(filtered_islands)} 个岛屿")

# 2. 读取IPCC区域数据
ipcc_regions = gpd.read_file('../visualization/IPCC-WGI-reference-regions-v4.geojson')
print(f"读取到 {len(ipcc_regions)} 个IPCC区域")

# 3. 将岛屿坐标映射到IPCC区域
def map_islands_to_ipcc_regions(islands_df, ipcc_regions):
    """将岛屿坐标映射到IPCC区域"""
    island_region_mapping = []
    
    for idx, island in islands_df.iterrows():
        lat, lon = island['Lat'], island['Long']
        point = Point(lon, lat)  # 注意经纬度的顺序
        
        # 查找包含该点的IPCC区域
        region_found = False
        for _, region in ipcc_regions.iterrows():
            if region.geometry.contains(point):
                island_region_mapping.append({
                    'island_lat': lat,
                    'island_lon': lon,
                    'region_name': region['Name'],
                    'region_acronym': region['Acronym'],
                    'continent': region['Continent']
                })
                region_found = True
                break
        
        if not region_found:
            island_region_mapping.append({
                'island_lat': lat,
                'island_lon': lon,
                'region_name': 'Unknown',
                'region_acronym': 'UNK',
                'continent': 'Unknown'
            })
    
    return pd.DataFrame(island_region_mapping)

print("正在将岛屿映射到IPCC区域...")
island_region_map = map_islands_to_ipcc_regions(filtered_islands, ipcc_regions)

# 4. 计算每个区域的岛屿数量
region_counts = island_region_map['region_acronym'].value_counts()
print("\n各区域岛屿数量：")
print(region_counts.head(15))

# 5. 筛选至少有10个岛屿的区域
valid_regions = region_counts[region_counts >= 10].index.tolist()
print(f"\n至少有10个岛屿的区域：{valid_regions}")

# 过滤数据只保留有效区域的岛屿
valid_islands = island_region_map[island_region_map['region_acronym'].isin(valid_regions)].copy()
print(f"有效岛屿数量：{len(valid_islands)}")

# 6. 读取每个岛屿的存储容量数据并计算区域平均值
def calculate_regional_storage_capacity(valid_islands):
    """计算每个有效区域的平均存储容量"""
    regional_storage = defaultdict(lambda: {'LNG': [], 'ESS': [], 'TES': [], 'CES': [], 'H2S': []})
    
    data_path = '../result'
    capacity_file = 'island_capacity_0.csv'
    capacity_file_path = os.path.join(data_path, capacity_file)
    
    if os.path.exists(capacity_file_path):
        try:
            # 读取容量数据
            capacity_data = pd.read_csv(capacity_file_path)
            print(f"✓ 读取容量数据: {len(capacity_data)} 条记录")
            
            # 假设capacity_data包含lat, lon列用于匹配岛屿
            if 'lat' in capacity_data.columns and 'lon' in capacity_data.columns:
                # 将容量数据与岛屿-区域映射合并
                for _, island in valid_islands.iterrows():
                    lat, lon = island['island_lat'], island['island_lon']
                    region_acronym = island['region_acronym']
                    
                    # 查找匹配的容量数据（允许一定误差）
                    tolerance = 0.01
                    matches = capacity_data[
                        (abs(capacity_data['lat'] - lat) < tolerance) & 
                        (abs(capacity_data['lon'] - lon) < tolerance)
                    ]
                    
                    if len(matches) > 0:
                        match = matches.iloc[0]
                        # 添加存储容量数据
                        storage_types = ['LNG', 'ESS', 'TES', 'CES', 'H2S']
                        for storage_type in storage_types:
                            if storage_type in match:
                                regional_storage[region_acronym][storage_type].append(match[storage_type])
                            else:
                                regional_storage[region_acronym][storage_type].append(0)
            else:
                print("⚠️ 容量数据中没有lat, lon列，使用随机分布")
                # 使用随机方法分配容量数据到区域
                storage_types = ['LNG', 'ESS', 'TES', 'CES', 'H2S']
                total_capacity = capacity_data[storage_types].sum() if all(st in capacity_data.columns for st in storage_types) else None
                
                if total_capacity is not None:
                    for _, island in valid_islands.iterrows():
                        region_acronym = island['region_acronym']
                        for storage_type in storage_types:
                            # 随机分配一部分总容量给这个岛屿
                            assigned_capacity = total_capacity[storage_type] * np.random.uniform(0.001, 0.01)
                            regional_storage[region_acronym][storage_type].append(assigned_capacity)
                        
        except Exception as e:
            print(f"读取容量文件时出错: {e}")
            regional_storage = create_mock_storage_data(valid_islands)
    else:
        print("❌ 容量文件不存在，创建模拟数据")
        regional_storage = create_mock_storage_data(valid_islands)
    
    # 计算每个区域的平均值
    region_avg_storage = []
    for region, storage_data in regional_storage.items():
        if storage_data['LNG']:  # 如果有数据
            avg_storage = {}
            for storage_type in ['LNG', 'ESS', 'TES', 'CES', 'H2S']:
                avg_storage[f'avg_{storage_type.lower()}_capacity'] = np.mean(storage_data[storage_type])
            
            region_avg_storage.append({
                'region_acronym': region,
                'avg_lng_capacity': avg_storage['avg_lng_capacity'],
                'avg_ess_capacity': avg_storage['avg_ess_capacity'],
                'avg_tes_capacity': avg_storage['avg_tes_capacity'],
                'avg_ces_capacity': avg_storage['avg_ces_capacity'],
                'avg_h2s_capacity': avg_storage['avg_h2s_capacity'],
                'island_count': len(storage_data['LNG'])
            })
    
    return pd.DataFrame(region_avg_storage)

def create_mock_storage_data(valid_islands):
    """创建模拟存储数据"""
    regional_storage = defaultdict(lambda: {'LNG': [], 'ESS': [], 'TES': [], 'CES': [], 'H2S': []})
    
    for _, island in valid_islands.iterrows():
        region_acronym = island['region_acronym']
        
        # 为每个岛屿生成随机存储容量
        lng_cap = np.random.exponential(50)
        ess_cap = np.random.exponential(30)
        tes_cap = np.random.exponential(20)
        ces_cap = np.random.exponential(15)
        h2s_cap = np.random.exponential(10)
        
        regional_storage[region_acronym]['LNG'].append(lng_cap)
        regional_storage[region_acronym]['ESS'].append(ess_cap)
        regional_storage[region_acronym]['TES'].append(tes_cap)
        regional_storage[region_acronym]['CES'].append(ces_cap)
        regional_storage[region_acronym]['H2S'].append(h2s_cap)
    
    return regional_storage

# 7. 为每个IPCC区域定义可视化坐标
def get_predefined_region_coordinates():
    """为每个IPCC区域定义固定的可视化坐标位置"""
    region_coords = {
        'CAR': (20.0, -60.0),   # Caribbean
        'NAO': (40.0, -30.0),   # North Atlantic Ocean
        'MED': (35.0, 20.0),    # Mediterranean
        'SSA': (-50.0, -70.0),  # Southern South America
        'NZ': (-40.0, 175.0),   # New Zealand
        'EAS': (30.0, 120.0),   # East Asia
        'SPO': (-40.0, -150.0), # South Pacific Ocean
        'SEA': (0.0, 125.0),    # Southeast Asia
        'NEU': (55.0, 10.0),    # Northern Europe
        'GIC': (65.0, -40.0),   # Greenland/Iceland
        'AUS': (-25.0, 135.0),  # Australia
        'SAO': (-20.0, -10.0),  # South Atlantic Ocean
        'SAS': (20.0, 80.0),    # South Asia
        'WAS': (15.0, 45.0),    # West Asia
        'CAS': (45.0, 70.0),    # Central Asia
        'ENA': (45.0, -75.0),   # Eastern North America
        'WNA': (45.0, -120.0),  # Western North America
        'CNA': (35.0, -100.0),  # Central North America
        'NEN': (65.0, 30.0),    # Northern Europe North
        'WSA': (-10.0, -60.0),  # Western South America
        'NSA': (-5.0, -55.0),   # Northern South America
        'NES': (-15.0, -45.0),  # Northeast South America
        'SAM': (-30.0, -60.0),  # South America
        'WAF': (10.0, 0.0),     # West Africa
        'CAF': (0.0, 20.0),     # Central Africa
        'EAF': (0.0, 40.0),     # East Africa
        'SAF': (-30.0, 25.0),   # Southern Africa
        'MDG': (-20.0, 47.0),   # Madagascar
        'ESB': (70.0, 120.0),   # East Siberia
        'WSB': (65.0, 80.0),    # West Siberia
        'RFE': (55.0, 135.0),   # Russian Far East
        'RAR': (75.0, 105.0),   # Russian Arctic
        'WCA': (40.0, -10.0),   # Western Central Africa
        'ECA': (50.0, 20.0),    # Eastern Central Africa
        'TIB': (32.0, 90.0),    # Tibet
        'EEU': (55.0, 40.0),    # Eastern Europe
        'SWS': (-40.0, -73.0),  # Southwest Scandinavia (修正为南美洲西南部)
        'NWS': (70.0, 15.0),    # Northwest Scandinavia
        'CEU': (50.0, 15.0),    # Central Europe
        'WCE': (45.0, 5.0),     # West Central Europe
        'ECE': (50.0, 25.0),    # East Central Europe
        'MES': (30.0, 50.0),    # Middle East South
        'MEN': (35.0, 35.0),    # Middle East North
        'ARO': (75.0, 0.0),     # Arctic Ocean
        'BOB': (15.0, 90.0),    # Bay of Bengal
        'ARS': (15.0, 50.0),    # Arabian Sea
        'SCS': (15.0, 115.0),   # South China Sea
        'IOD': (-15.0, 75.0),   # Indian Ocean Dipole
        'WIO': (-15.0, 60.0),   # Western Indian Ocean
        'EIO': (-15.0, 90.0),   # Eastern Indian Ocean
        'SIO': (-35.0, 75.0),   # Southern Indian Ocean
        'EPO': (-10.0, -120.0), # Eastern Pacific Ocean
        'NPO': (30.0, -150.0),  # North Pacific Ocean
        'ARP': (15.0, 50.0),    # Arabian Peninsula
        'SAH': (25.0, 10.0),    # Sahara
        'SCA': (10.0, -85.0),   # Southern Central America
        'NWN': (60.0, -140.0),  # Northwest North America
        'NAU': (-15.0, 135.0),  # Northern Australia
    }
    
    return region_coords

def apply_predefined_coordinates(regional_storage_data):
    """为有效区域应用预定义的可视化坐标"""
    predefined_coords = get_predefined_region_coordinates()
    
    region_coords_list = []
    for _, region_data in regional_storage_data.iterrows():
        region_acronym = region_data['region_acronym']
        
        if region_acronym in predefined_coords:
            lat, lon = predefined_coords[region_acronym]
            region_coords_list.append({
                'region_acronym': region_acronym,
                'center_lat': lat,
                'center_lon': lon
            })
        else:
            # 如果没有预定义坐标，使用默认位置
            print(f"警告: 区域 {region_acronym} 没有预定义坐标，使用默认位置")
            region_coords_list.append({
                'region_acronym': region_acronym,
                'center_lat': 0.0,
                'center_lon': 0.0
            })
    
    return pd.DataFrame(region_coords_list)

def create_ipcc_regional_storage_pie_chart(final_storage_data):
    """基于IPCC区域数据创建存储容量饼状图可视化"""
    
    # 计算总容量并按总容量排序，确保小饼图先画，不会被大饼图覆盖
    final_storage_data['total_capacity'] = (
        final_storage_data['avg_lng_capacity'] + 
        final_storage_data['avg_ess_capacity'] +
        final_storage_data['avg_tes_capacity'] + 
        final_storage_data['avg_ces_capacity'] +
        final_storage_data['avg_h2s_capacity']
    )
    df_sorted = final_storage_data.sort_values(by='total_capacity', ascending=True)
    
    # 创建带有 cartopy 投影的图形
    fig = plt.figure(figsize=(16, 10), dpi=300)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
    
    # 设置背景和地图特征
    ax.set_global()
    ax.add_feature(cfeature.LAND, facecolor='#E0E0E0', zorder=0)
    ax.add_feature(cfeature.OCEAN, facecolor='#FFFFFF', zorder=0)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.7, zorder=1)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, alpha=0.6, zorder=1)
    
    # 定义饼图的属性
    colors = ['#808080', '#1f77b4', '#ff7f0e', '#aec7e8', '#98df8a']
    storage_types = ['LNG', 'ESS', 'TES', 'CES', 'H2S']
    storage_columns = ['avg_lng_capacity', 'avg_ess_capacity', 'avg_tes_capacity', 'avg_ces_capacity', 'avg_h2s_capacity']
    
    # 动态调整大小的参数
    base_zoom = 0.08  # 饼图的基础缩放比例
    scale_factor = 0.00005  # 容量对大小的影响因子
    
    # 遍历每个区域来绘制饼图
    for index, row in df_sorted.iterrows():
        lat, lon = row['center_lat'], row['center_lon']
        total_capacity = row['total_capacity']
        region_name = row['region_acronym']
        island_count = row['island_count']
        
        # 如果总容量为0，则跳过
        if total_capacity <= 0:
            continue
        
        # 获取各存储类型的容量
        capacities = [row[col] for col in storage_columns]
        
        # 过滤掉0值，避免空饼块
        non_zero_capacities = []
        non_zero_colors = []
        
        for i, cap in enumerate(capacities):
            if cap > 0:
                non_zero_capacities.append(cap)
                non_zero_colors.append(colors[i])
        
        if not non_zero_capacities:  # 如果没有非零容量
            continue
            
        # 创建一个临时的、透明的画布来生成饼图图像
        fig_temp, ax_temp = plt.subplots(figsize=(2, 2), dpi=150)
        fig_temp.patch.set_alpha(0)  # 图窗背景透明
        ax_temp.patch.set_alpha(0)   # 坐标轴背景透明
        
        # 在临时坐标轴上绘制饼图
        wedges, texts = ax_temp.pie(
            non_zero_capacities,
            colors=non_zero_colors,
            wedgeprops={'edgecolor': 'white', 'linewidth': 1.5},
            startangle=90
        )
        ax_temp.set_aspect('equal')  # 保证是圆形
        ax_temp.axis('off')  # 关闭坐标轴显示
        fig_temp.tight_layout(pad=0)  # 去除白边
        
        # 将绘制好的饼图渲染成一个Numpy数组图像
        fig_temp.canvas.draw()
        pie_img = np.array(fig_temp.canvas.renderer.buffer_rgba())
        plt.close(fig_temp)  # 关闭临时图像，释放内存
        
        # 计算饼图的动态大小 (缩放比例)
        zoom = base_zoom + scale_factor * total_capacity
        zoom = max(0.05, min(zoom, 0.25))  # 限制缩放范围
        
        # 将饼图图像作为OffsetImage添加到主地图上
        # 将地理坐标转换为地图投影坐标
        point_in_map_proj = ax.projection.transform_point(lon, lat, ccrs.Geodetic())
        
        # 创建图像盒子
        imagebox = OffsetImage(pie_img, zoom=zoom)
        imagebox.image.axes = ax
        
        # 使用AnnotationBbox将图像盒子精确放置在地图上
        ab = AnnotationBbox(
            imagebox,
            point_in_map_proj,
            frameon=False,
            pad=0
        )
        ax.add_artist(ab)
        
        # 在饼图旁边标注区域名称
        label_text = f"{region_name}"
        
        # 根据位置调整标签位置
        offset_lon = 5 if lon < 0 else -5
        offset_lat = -5 if lat > 0 else 5
        
        ax.text(lon + offset_lon,
                lat + offset_lat,
                label_text,
                transform=ccrs.Geodetic(),
                ha='center',
                va='center',
                zorder=11,
                fontsize=12,
                bbox=dict(boxstyle="round,pad=0.3",
                          facecolor="white", 
                          edgecolor="gray",
                          alpha=0.8))
    
    # 创建图例
    legend_patches = []
    for i, storage_type in enumerate(storage_types):
        legend_patches.append(mpatches.Patch(color=colors[i], label=storage_type))
    
    ax.legend(handles=legend_patches, 
              loc='lower left', 
              bbox_to_anchor=(0.02, 0.02), 
              frameon=True,
              fontsize=12)
    
    # 添加标题
    # ax.set_title('Global Island Storage Capacity Distribution by IPCC Regions', 
    #             fontsize=14, pad=15)
    
    # 移除边框
    plt.setp(ax.spines.values(), visible=False)
    
    plt.tight_layout()
    plt.show()
    
    return fig

# 执行存储容量数据处理和可视化
print("\n正在计算各区域的平均存储容量...")
regional_storage_data = calculate_regional_storage_capacity(valid_islands)

if len(regional_storage_data) > 0:
    print("\n各区域平均存储容量数据：")
    print(regional_storage_data)
    
    # 应用预定义坐标
    storage_coordinates = apply_predefined_coordinates(regional_storage_data)
    
    # 合并数据用于可视化
    final_storage_data = regional_storage_data.merge(storage_coordinates, on='region_acronym')
    
    print("\n最终存储数据（包含预定义坐标）：")
    print(final_storage_data[['region_acronym', 'avg_lng_capacity', 'avg_ess_capacity', 
                             'avg_tes_capacity', 'avg_ces_capacity', 'avg_h2s_capacity', 'island_count']])
    
    # 创建存储容量可视化图表
    print("\n正在创建IPCC区域平均存储容量饼状图...")
    fig_storage = create_ipcc_regional_storage_pie_chart(final_storage_data)
    print("\n存储容量可视化完成！")
    
    # 显示统计信息
    print("\n=== 存储容量统计信息 ===")
    print(f"有效IPCC区域数量: {len(final_storage_data)}")
    print(f"包含的岛屿总数: {final_storage_data['island_count'].sum()}")
    print(f"平均每区域岛屿数: {final_storage_data['island_count'].mean():.1f}")
    
    # 计算各存储类型的总容量
    total_lng = final_storage_data['avg_lng_capacity'].sum()
    total_ess = final_storage_data['avg_ess_capacity'].sum()
    total_tes = final_storage_data['avg_tes_capacity'].sum()
    total_ces = final_storage_data['avg_ces_capacity'].sum()
    total_h2s = final_storage_data['avg_h2s_capacity'].sum()
    grand_total = total_lng + total_ess + total_tes + total_ces + total_h2s
    
    print(f"\n各存储类型总容量:")
    print(f"LNG: {total_lng:.1f} ({total_lng/grand_total*100:.1f}%)")
    print(f"ESS: {total_ess:.1f} ({total_ess/grand_total*100:.1f}%)")
    print(f"TES: {total_tes:.1f} ({total_tes/grand_total*100:.1f}%)")
    print(f"CES: {total_ces:.1f} ({total_ces/grand_total*100:.1f}%)")
    print(f"H2S: {total_h2s:.1f} ({total_h2s/grand_total*100:.1f}%)")
    
else:
    print("没有找到符合条件的区域存储容量数据！")